In [ ]:
!nvidia-smi

In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import pandas as pd
import numpy as np
import re
import librosa
import seaborn as sns
import os
import json
import IPython.display as ipd
import soundfile as sf
import torch
import h5py
import onnxruntime as ort
import openvino as ov
import torch.quantization.quantize_fx as quantize_fx

from glob import glob
from tqdm import tqdm
from pprint import pprint
from matplotlib import pyplot as plt
from itertools import chain
from os.path import join as pjoin
from torchaudio.transforms import AmplitudeToDB, MelSpectrogram
from copy import deepcopy
from sklearn.metrics import f1_score 

# from code_base.utils import parallel_librosa_load, groupby_np_array, stack_and_max_by_samples, macro_f1_similarity, N_CLASSES_2021_2022, N_CLASSES_2021, comp_metric, N_CLASSES_XC_LIGIT_SHORTEN, N_CLASSES_XC_LIGIT_EVEN_SHORTEN
# from code_base.utils.constants import SAMPLE_RATE
from code_base.utils.onnx_utils import ONNXEnsemble, convert_to_onnx
from code_base.models import WaveCNNClasifier, WaveCNNAttenClasifier, WaveTDNNClasifier
from code_base.datasets import WaveDataset, WaveAllFileDataset
from code_base.utils.swa import avarage_weights, delete_prefix_from_chkp
from code_base.inefernce import BirdsInference
from code_base.utils import load_json, compose_submission_dataframe, groupby_np_array, stack_and_max_by_samples, write_json
from code_base.utils.metrics import score_numpy
%matplotlib inline


# Clean logdirs

In [ ]:
# def filt_criteria(input):
#     num_folds = sum("fold_" in el for el in glob(pjoin(input, "*")))
#     if num_folds > 0 and num_folds < 5:
#         print(input, num_folds)
#         return False
#     else:
#         return True
    
# all_logdirs = glob("../logdirs/*")
# logdirs_to_remove = [el for el in all_logdirs if not filt_criteria(el)]

In [ ]:
# for el in logdirs_to_remove[:-1]:
#     !rm {el} -rf

# Config

In [ ]:
!ls -lt ../logdirs/convnextv2_tiny_fcmae_ft_in22k_in1k_384_Exp_noamp_64bs_5sec_PrevCompXCScoredDataNoSecLab_BackGroundSoundScapeP05_mixupP05_RandomFiltering_balancedSampler_Radamlr1e4_CosBatchLR1e6_Epoch30_BCELoss_*

In [ ]:
# print("Possible exps:\n\n{}".format("\n".join(os.listdir("../logdirs/"))))

In [ ]:
bird2id_source = load_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024_scoed_add_data.json")
bird2id_target = load_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024.json")

id2bird_source = {v:k for k,v in bird2id_source.items()}
id2bird_target = {v:k for k,v in bird2id_target.items()}

REARRANGE_INDICES = np.array([
    bird2id_source[id2bird_target[i]] for i in range(len(id2bird_target))
]).astype(int)

def prune_checkpoint_rule(inp_chkp):
    inp_chkp["head.attention.weight"] = inp_chkp["head.attention.weight"][REARRANGE_INDICES]
    inp_chkp["head.attention.bias"] = inp_chkp["head.attention.bias"][REARRANGE_INDICES]
    
    inp_chkp["head.fix_scale.weight"] = inp_chkp["head.fix_scale.weight"][REARRANGE_INDICES]
    inp_chkp["head.fix_scale.bias"] = inp_chkp["head.fix_scale.bias"][REARRANGE_INDICES]

    return inp_chkp

In [ ]:
EXP_NAME = "tf_efficientnetv2_b1_in1k_Exp_noamp_FixedAmp2Db_64bs_5sec_TimeFlip05_FormixupAlpha05NormedBinTgtEqW_Radamlr1e3_CosBatchLR1e6_Epoch30_SpecAugV207_FocalBCELoss_5Folds_NoDuplsV1"
TRAIN_PERIOD = 5
print("Possible checkpoints:\n\n{}".format("\n".join(set([os.path.basename(el) for el in glob(f"../logdirs/{EXP_NAME}/*/checkpoints/*.ckpt*") if "train" not in os.path.basename(el)]))))

In [ ]:
conf_path = glob(f"../logdirs/{EXP_NAME}/code/*train_configs*.py")
assert len(conf_path) == 1
conf_path = conf_path[0]
!cat {conf_path}

In [ ]:
CONFIG = {
    # Main
    "run_validation": True,
    "run_test": False,
    # Inference Class
    "use_sigmoid": False,
    "use_compiled_fp16": False,
    "aggregate_preds": True,
    # Data config
    "train_df_path": "/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended_noduplv1.csv",
    "split_path": "/home/vova/data/exps/birdclef_2024/cv_splits/birdclef_2024_5_folds_split_nodupl.npy",
    "n_folds":5,
    # "n_folds": 1,
    "train_data_root":"/home/vova/data/exps/birdclef_2024/birdclef_2024/train_audio/",
    "test_data_root":"/home/vova/data/exps/birdclef_2024/birdclef_2024/unlabeled_soundscapes/*.ogg",
    "label_map_data_path": "/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024.json",
    "scored_birds_path":"/home/vova/data/exps/birdclef_2024/scored_birds/sb_2024.json", 
    "lookback":None,
    "lookahead":None,
    "segment_len":5,
    "step": None,
    "late_normalize": True,
    # Model config
    "exp_name":EXP_NAME,
    "model_class": WaveCNNAttenClasifier,
    "model_config": dict(
        backbone="tf_efficientnetv2_b1.in1k",
        mel_spec_paramms={
            "sample_rate": 32000,
            "n_mels": 128,
            "f_min": 20,
            "n_fft": 2048,
            "hop_length": 512,
            "normalized": True,
        },
        head_config={
            "p": 0.5,
            "num_class": 188,
            "train_period": TRAIN_PERIOD,
            "infer_period": TRAIN_PERIOD,
            "output_type": "clipwise_pred_long",
        },
        exportable=True,
        fixed_amplitude_to_db=True
    ),
    "chkp_name":"last.ckpt",
    "swa_checkpoint_regex": r'(?P<key>\w+)=(?P<value>[\d.]+)(?=\.ckpt|$)',
    "use_sigmoid": True,
    "swa_sort_rule": lambda x: -float(x["valid_roc_auc"]),
    "delete_prefix": "model.",
    "n_swa_models": 3,
    "model_output_key": None,
    # "prune_checkpoint_func": prune_checkpoint_rule,

    # Compilation config
    "use_openvino": True,
    "use_fp16": True,
    "folds_to_onnx": [0, 1, 2, 3, 4],
    "final_activation": "softmax"
}

if CONFIG.get("use_sed_mode", False):
    assert CONFIG["step"] is not None
else:
    assert CONFIG["step"] is None
    
if "folds" not in CONFIG:
    CONFIG["folds"] = list(range(CONFIG["n_folds"]))
    

# Data

In [ ]:
bird2id = load_json(CONFIG["label_map_data_path"])

In [ ]:
if CONFIG["run_validation"]:
    df = pd.read_csv(CONFIG["train_df_path"])
    split = np.load(CONFIG["split_path"], allow_pickle=True)
    val_df = [df.iloc[split[i][1]].reset_index(drop=True) for i in CONFIG["folds"]]

In [ ]:
if CONFIG["run_test"]:
    test_au_pathes = glob(CONFIG["test_data_root"])[:20]

    test_df = pd.DataFrame({
        "filename": test_au_pathes,
        "duration_s": [librosa.get_duration(filename=el) for el in test_au_pathes]
    })

In [ ]:
if CONFIG["run_validation"]:
    val_ds_conig = {
       "root": CONFIG["train_data_root"],
       "label_str2int_mapping_path": CONFIG["label_map_data_path"],
       "use_audio_cache": True,
       "n_cores": 64,
       "verbose": False,
       "segment_len": CONFIG["segment_len"],
       "lookback":CONFIG["lookback"],
       "lookahead":CONFIG["lookahead"],
       "sample_id": None,
       "late_normalize": CONFIG["late_normalize"],
       "step": CONFIG["step"],
        "validate_sr": 32_000
    }
if CONFIG["run_test"]:
    ds_config_test = {
       "root": "",
       "label_str2int_mapping_path": CONFIG["label_map_data_path"],
       "n_cores": 64,
       "use_audio_cache": True,
       "test_mode": True,
       "segment_len": CONFIG["segment_len"],
       "lookback":CONFIG["lookback"],
       "lookahead":CONFIG["lookahead"],
        "sample_id": None,
        "late_normalize": CONFIG["late_normalize"],
        "step": CONFIG["step"],
        "validate_sr": 32_000
    }
loader_config = {
    "batch_size": 64,
    "drop_last": False,
    "shuffle": False,
    "num_workers": 0,
}

In [ ]:
if CONFIG["run_test"]:
    ds_test = WaveAllFileDataset(df=test_df, **ds_config_test)
if CONFIG["run_validation"]:
    ds_val = [WaveAllFileDataset(df=df, **val_ds_conig) for df in val_df]

In [ ]:
if CONFIG["run_validation"]:
    loader_val = [torch.utils.data.DataLoader(
        ds,
        **loader_config,
    )for ds in ds_val]
if CONFIG["run_test"]:
    loader_test = torch.utils.data.DataLoader(
        ds_test,
        **loader_config,
    )

In [ ]:
len(loader_val)

# Model

In [ ]:
def create_model_and_upload_chkp(
    model_class,
    model_config,
    model_device,
    model_chkp_root,
    model_chkp_basename=None,
    model_chkp_regex=None,
    delete_prefix=None,
    swa_sort_rule=None,
    n_swa_to_take=3,
    prune_checkpoint_func=None
):
    if model_chkp_basename is None:
        basenames = os.listdir(model_chkp_root)
        checkpoints = []
        for el in basenames:
            matches = re.findall(model_chkp_regex, el)
            if not matches:
                continue
            parsed_dict = {key: value for key, value in matches}
            parsed_dict["name"] = el
            checkpoints.append(parsed_dict)
        print("SWA checkpoints")
        pprint(checkpoints)
        checkpoints = sorted(checkpoints, key=swa_sort_rule)
        checkpoints = checkpoints[:n_swa_to_take]
        print("SWA sorted checkpoints")
        pprint(checkpoints)
        if len(checkpoints) > 1:
            checkpoints = [
                torch.load(os.path.join(model_chkp_root, el["name"]), map_location="cpu")["state_dict"] for el in checkpoints
            ]
            t_chkp = avarage_weights(
                nn_weights=checkpoints,
                delete_prefix=delete_prefix
            )
        else:
            chkp_path = os.path.join(model_chkp_root, checkpoints[0]["name"])
            print("vanilla model")
            print("Loading", chkp_path)
            t_chkp = torch.load(
                chkp_path, 
                map_location="cpu"
            )["state_dict"]
            if delete_prefix is not None:
                t_chkp = delete_prefix_from_chkp(t_chkp, delete_prefix)
    else:
        chkp_path = os.path.join(model_chkp_root, model_chkp_basename)
        print("vanilla model")
        print("Loading", chkp_path)
        t_chkp = torch.load(
            chkp_path, 
            map_location="cpu"
        )["state_dict"]
        if delete_prefix is not None:
            t_chkp = delete_prefix_from_chkp(t_chkp, delete_prefix)

    if prune_checkpoint_func is not None:
        t_chkp = prune_checkpoint_func(t_chkp)
    t_model = model_class(**model_config, device=model_device) 
    print("Missing keys: ", set(t_model.state_dict().keys()) - set(t_chkp))
    print("Extra keys: ",  set(t_chkp) - set(t_model.state_dict().keys()))
    t_model.load_state_dict(t_chkp, strict=False)
    t_model.eval()
    return t_model

In [ ]:

model = [create_model_and_upload_chkp(
    model_class=CONFIG["model_class"],
    model_config=CONFIG['model_config'],
    model_device="cuda",
    model_chkp_root=f"../logdirs/{CONFIG['exp_name']}/fold_{m_i}/checkpoints",
    # model_chkp_root=f"../logdirs/{CONFIG['exp_name']}/checkpoints",
    model_chkp_basename=CONFIG["chkp_name"] if CONFIG["swa_checkpoint_regex"] is None else None,
    model_chkp_regex=CONFIG.get("swa_checkpoint_regex"),
    swa_sort_rule=CONFIG.get("swa_sort_rule"),
    n_swa_to_take=CONFIG.get("n_swa_models", 3),
    delete_prefix=CONFIG.get("delete_prefix"),
    prune_checkpoint_func=CONFIG.get("prune_checkpoint_func")
) for m_i in range(CONFIG["n_folds"])]


In [ ]:
# !rm ../logdirs/convnextv2_tiny_fcmae_ft_in22k_in1k_384_Exp_noamp_64bs_5sec_mixupP05_balancedSampler_lr1e4_CosineLREpoch50_ValV2_202xXcAddDataNoAddSecLabels_BackGroundSoundScapeP05_FocalLoss_FromPretrainFinalV1/onnx_ensem_3first_folds -rf

In [ ]:
# if CONFIG.get("folds_to_onnx") is not None:
#     if isinstance(CONFIG["folds_to_onnx"], int):
        
#         exportable_ensem = ONNXEnsemble(
#             model_class=CONFIG["model_class"],
#             configs=[deepcopy(CONFIG['model_config']) for _ in CONFIG["folds"][:CONFIG["folds_to_onnx"]]],
#             final_activation=CONFIG.get("final_activation", None)
#         )
#         for model_id in range(len(exportable_ensem.models)):
#             exportable_ensem.models[model_id].load_state_dict(model[model_id].state_dict())
#         exportable_ensem.eval()
#         convert_to_onnx(
#             model_to_convert=exportable_ensem,
#             sample_input=torch.randn(5, TRAIN_PERIOD * 32_000),
#             base_path=f"../logdirs/{CONFIG['exp_name']}/onnx_ensem_{CONFIG['folds_to_onnx']}first_folds",
#             use_fp16=CONFIG.get("use_fp16", False),
#             use_openvino=CONFIG.get("use_openvino", False)
#         )

#     elif isinstance(CONFIG["folds_to_onnx"], list):
#         for fold_id in CONFIG["folds_to_onnx"]:
#             exportable_ensem = ONNXEnsemble(
#                 model_class=CONFIG["model_class"],
#                 configs=[deepcopy(CONFIG['model_config'])],
#                 final_activation=CONFIG.get("final_activation", None)
#             )
#             # for model_id in range(len(exportable_ensem.models)):
#             exportable_ensem.models[0].load_state_dict(model[fold_id].state_dict())
            
#             exportable_ensem.eval()
#             convert_to_onnx(
#                 model_to_convert=exportable_ensem,
#                 sample_input=torch.randn(5, TRAIN_PERIOD * 32_000),
#                 base_path=f"../logdirs/{CONFIG['exp_name']}/onnx_ensem_fold{fold_id}",
#                 use_fp16=CONFIG.get("use_fp16", False),
#                 use_openvino=CONFIG.get("use_openvino", False)
#             )

#     else:
#         raise ValueError(f"{type(CONFIG['folds_to_onnx'])} - unsupported type")

# Inference Class

In [ ]:
inference_class = BirdsInference(
    device="cuda",
    verbose_tqdm=True,
    use_sigmoid=CONFIG["use_sigmoid"],
    model_output_key=CONFIG["model_output_key"],
    aggregate_preds=CONFIG.get("aggregate_preds", True)
)

# Val Predict

In [ ]:
if CONFIG["run_validation"]:
    val_tgts, val_preds = inference_class.predict_val_loaders(
        nn_models=model,
        data_loaders=loader_val
    )
    if CONFIG.get("scored_birds_path", None):
        print("Extracting scored_bird_ids indices")
        scored_bird = load_json(CONFIG["scored_birds_path"])
        scored_bird_ids = [bird2id[el] for el in scored_bird]
        val_tgts = [el[:,scored_bird_ids] for el in val_tgts]
        val_preds = [el[:,scored_bird_ids] for el in val_preds]
    cmaps = [
        score_numpy(gt, pr) for gt, pr in zip(val_tgts, val_preds)
    ]

    print(
        f"Folds Roc Auc: {cmaps}\n"
        f"Mean Roc Auc: {np.mean(cmaps)}"
    )

In [ ]:
os.makedirs(os.path.join("../predictions/", EXP_NAME))
np.save(
    os.path.join("../predictions/", EXP_NAME, "tgts.npy"),
    np.array(val_tgts, dtype=object)
)
np.save(
    os.path.join("../predictions/", EXP_NAME, "preds.npy"),
    np.array(val_preds, dtype=object)
)

In [ ]:
# core = ov.Core()
# model = core.read_model(model=f"../logdirs/{CONFIG['exp_name']}/onnx_ensem_fold0_openvino_fp16/model_simpl.xml")
# compiled_model = core.compile_model(model=model, device_name="CPU")

In [ ]:
# if CONFIG["run_validation"]:
#     val_tgts, val_preds = inference_class.predict_val_loaders(
#         nn_models=[compiled_model],
#         data_loaders=loader_val,
#         is_openvino_model=True
#     )
#     if CONFIG.get("scored_birds_path", None):
#         print("Extracting scored_bird_ids indices")
#         scored_bird = load_json(CONFIG["scored_birds_path"])
#         scored_bird_ids = [bird2id[el] for el in scored_bird]
#         val_tgts = [el[:,scored_bird_ids] for el in val_tgts]
#         val_preds = [el[:,scored_bird_ids] for el in val_preds]
#     cmaps = [
#         score_numpy(gt, pr) for gt, pr in zip(val_tgts, val_preds)
#     ]

#     print(
#         f"Folds Roc Auc: {cmaps}\n"
#         f"Mean Roc Auc: {np.mean(cmaps)}"
#     )

In [ ]:
print(
    EXP_NAME, "\n"
    f"Folds Roc Auc: {cmaps}\n"
    f"Mean Roc Auc: {np.mean(cmaps)}\n"
    f"OOF Roc Auc: {score_numpy(np.concatenate(val_tgts), np.concatenate(val_preds))}"
)

In [ ]:
val_preds[0].max(axis=1)

# Search for Pseudo

In [ ]:
all_val_tgts = np.concatenate(val_tgts)
all_val_preds = np.concatenate(val_preds)

In [ ]:
all_treshes = np.arange(0.05 , 1.0 + 0.05, 0.05)
all_scores = []
for tresh in tqdm(all_treshes):
    all_scores.append(f1_score(
        all_val_tgts,
        all_val_preds > tresh,
        average="macro"
    ))

In [ ]:
best_idx = np.argmax(all_scores)
best_tresh, best_score = all_treshes[best_idx], all_scores[best_idx]

print(f"Best Score = {best_score} and is reached on {best_tresh}")

plt.title("Macro F1 depending on tresh")
plt.plot(all_treshes, all_scores)
plt.show()

# Test Pred

In [ ]:
if CONFIG["run_test"]:
    test_preds, test_preds_long, test_dfidx, test_end = inference_class.predict_test_loader(
        nn_models=model,
        data_loader=loader_test
    )
    test_pred_df = compose_submission_dataframe(
        probs=test_preds,
        dfidxs=test_dfidx,
        end_seconds=test_end,
        filenames=loader_test.dataset.df[loader_test.dataset.name_col].copy(),
        bird2id=bird2id
    )

In [ ]:
# test_pred_df

# ONNX Test Pred

In [ ]:
inference_class_onnx = BirdsInference(
    device="cpu",
    verbose_tqdm=True,
    use_sigmoid=CONFIG["use_sigmoid"],
    model_output_key=CONFIG["model_output_key"],
    use_compiled_fp16=CONFIG["use_compiled_fp16"],
)
# onnx_model = ort.InferenceSession(
#     # f"../logdirs/{CONFIG['exp_name']}/onnx_ensem_2first_folds/model_simpl.onnx"
#     "../logdirs/convnext_small_fb_in22k_ft_in1k_384_Exp_noamp_64bs_5sec_mixupP05_RandomFiltering_balancedSampler_Radamlr1e4_CosBatchLR1e6_Epoch50_SpecAugV1_FocalLoss_Full/onnx_ensem/model_simpl.onnx"
# )
# core = ov.Core()
# model = core.read_model(model=f"../logdirs/convnext_small_fb_in22k_ft_in1k_384_Exp_noamp_64bs_5sec_mixupP05_RandomFiltering_balancedSampler_Radamlr1e4_CosBatchLR1e6_Epoch50_SpecAugV1_FocalLoss_Full/openvino_ensem/model_simpl.xml")
# compiled_model = core.compile_model(model=model, device_name="CPU")

In [ ]:
test_preds_onnx, test_dfidx_onnx, test_end_onnx = inference_class_onnx.predict_test_loader(
    nn_models=compiled_model,
    data_loader=loader_test,
    is_openvino_model=True,
)

In [ ]:
test_pred_df_onnx = compose_submission_dataframe(
    probs=test_preds_onnx,
    dfidxs=test_dfidx_onnx,
    end_seconds=test_end_onnx,
    filenames=loader_test.dataset.df[loader_test.dataset.name_col].copy(),
    bird2id=bird2id
)

In [ ]:
test_pred_df_onnx.iloc[:,1:].values

In [ ]:
test_pred_df_onnx_fp16 = pd.read_csv("temp_fp16.csv")

In [ ]:
test_pred_df_onnx_fp16.iloc[:,1:].values

In [ ]:
np.abs(test_pred_df_onnx_fp16.iloc[:,1:].values - test_pred_df_onnx.iloc[:,1:].values).max()

In [ ]:
# test_pred_df_onnx.to_csv("temp_fp16.csv", index=False)

In [ ]:
plt.hist(test_pred_df_onnx.iloc[:,1:].values.flatten())

In [ ]:
if CONFIG["run_test"]:
    inference_class_onnx = BirdsInference(
        device="cpu",
        verbose_tqdm=True,
        use_sigmoid=CONFIG["use_sigmoid"],
        model_output_key=CONFIG["model_output_key"],
        
    )
    onnx_model = ort.InferenceSession(
        f"../logdirs/{CONFIG['exp_name']}/onnx_ensem_2first_folds/model_simpl.onnx"
    )
    test_preds_onnx, test_preds_long_onnx, test_dfidx_onnx, test_end_onnx = inference_class_onnx.predict_test_loader(
        nn_models=onnx_model,
        data_loader=loader_test,
        is_onnx_model=True
    )
    test_pred_df_onnx = compose_submission_dataframe(
        probs=test_preds_onnx,
        dfidxs=test_dfidx_onnx,
        end_seconds=test_end_onnx,
        filenames=loader_test.dataset.df[loader_test.dataset.name_col].copy(),
        bird2id=bird2id
    )

In [ ]:
test_pred_df_onnx

# Map Predictions

In [ ]:
if CONFIG.get("check_inf_class", False):
    val_ds_check = WaveAllFileDataset(df=val_df[0], **{
           "root": CONFIG["train_data_root"],
           "label_str2int_mapping_path": CONFIG["label_map_data_path"],
           "n_cores": 64,
           "use_audio_cache": True,
           "test_mode": True,
           "segment_len": CONFIG["segment_len"],
           "lookback":CONFIG["lookback"],
           "lookahead":CONFIG["lookahead"],
            "sample_id": None,
            "late_normalize": CONFIG["late_normalize"],
            "step": CONFIG["step"],
        }
    )
    val_loader_check = torch.utils.data.DataLoader(
        val_ds_check,
        **loader_config
    )
    
    test_preds_check, test_preds_long_check, test_dfidx_check, test_end_check = inference_class.predict_test_loader(
        nn_models=model,
        data_loader=val_loader_check
    )
    test_preds_check_grouped = groupby_np_array(
        groupby_f=test_dfidx_check,
        array_to_group=test_preds_check,
        apply_f=stack_and_max_by_samples,
    )
    print(np.allclose(
        test_preds_check_grouped,
        val_preds
    ))